# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [1]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from sphinx_astrorefs import pybtex_astro
from time import strptime
import string
import html
import os
import re

In [2]:
pybtex_astro.register()

In [3]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {

    "journal":{
        "file": "export-bibtex.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    } 
}

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [5]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields
        
        try:
            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-").replace("$_ensuremathodot$","")
            print(clean_title)

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

            #Build Citation from text
            citation = ""

            #citation authors - todo - add highlighting for primary author?
            for author in bibdata.entries[bib_id].persons["author"]:
                if author.first_names and author.last_names:
                    citation = citation+" "+author.first_names[0].encode().decode('latex').replace('{', '').replace('}', '')+" "+author.last_names[0].encode().decode('latex').replace('{', '').replace('}', '')+", "

            #citation title
            citation = citation + "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","").replace("$_ensuremathodot$","<sub>☉</sub>") ) + ".\""

            #add venue logic depending on citation type
            venue = publist[pubsource]["venue-pretext"]+b[publist[pubsource]["venuekey"]].replace("{", "").replace("}","").replace("\\","")
            if venue == "mnras":
                venue ="Monthly Notices of the Royal Astronomical Society"
            elif venue == "aj":
                venue ="The Astronomical Journal"
            elif venue == "aap":
                venue ="Astronomy & Astrophysics"
            elif venue == "nat":
                venue ="Nature"
            elif venue == "apj":
                venue ="The Astrophysical Journal"
            elif venue == "apjl":
                venue ="The Astrophysical Journal Letters"
            elif venue == "actaa":
                venue ="Acta Astronomica"
            elif venue == "pasp":
                venue ="Publications of the ASP"               

            citation = citation + " " + html_escape(venue)
            citation = citation + ", " + pub_year + "."

            
            ## YAML variables
            md = "---\ntitle: \""   + html_escape( b["title"].replace("{", "").replace("}","").replace("\\","").replace("$_ensuremathodot$","<sub>☉</sub>") ) + '"\n'
            
            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            md += "\nvenue: '" + html_escape(venue) + "'"
            
            url = False
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    md += "\npaperurl: '" + b["url"] + "'"
                    url = True

            md += "\ncitation: '" + html_escape(citation) + "'"

            md += "\n---"

            
            ## Markdown description for individual page
            if note:
                md += "\n" + html_escape(b["note"]) + "\n"

            if url:
                md += "\n[Access paper here](" + b["url"] + "){:target=\"_blank\"}\n" 
            else:
                md += "\nUse [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+"){:target=\"_blank\"} for full citation"

            md_filename = os.path.basename(md_filename)

            with open("/home/mrabus/Jupyter_NBs/MarkusRabus.github.io/_publications/" + md_filename, 'w') as f:
                f.write(md)
            print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


Deep-drilling-in-the-time-domain-with-DECam:-Survey-characterization
SUCESSFULLY PARSED 2022MNRAS.tmp.3146G: " {Deep drilling in the time domain with DECam: Survey charact ... "
Updated-orbital-monitoring-and-dynamical-masses-for-nearby-M-dwarf-binaries
SUCESSFULLY PARSED 2022A&A...666A..16C: " {Updated orbital monitoring and dynamical masses for nearby  ... "
Physical-properties-of-near-Earth-asteroid-(2102)-Tantalus-from-multiwavelength-observations
SUCESSFULLY PARSED 2022MNRAS.515.4551R: " {Physical properties of near-Earth asteroid (2102) Tantalus  ... "
A-search-for-transit-timing-variations-in-the-HATS-18-planetary-system
SUCESSFULLY PARSED 2022MNRAS.515.3212S: " {A search for transit timing variations in the HATS-18 plane ... "
Near-infrared-and-optical-emission-of-WASP-5-b
SUCESSFULLY PARSED 2022A&A...664A..47K: " {Near-infrared and optical emission of WASP-5 b}  "
A-Possible-Alignment-Between-the-Orbits-of-Planetary-Systems-and-their-Visual-Binary-Companions
SUCESSFULLY PARSED

In [6]:
author.last_names[0].encode().decode('latex').replace('{', '').replace('}', '')

'Mansfield'

In [7]:
if author.first_names and author.last_names:
    print('True')

True


In [8]:
autor= bibdata.entries['2020AJ....160..235B'].persons["author"]

In [9]:
autor[5].first_names[0].encode().decode('latex').replace('{', '').replace('}', '')

'Néstor'

In [10]:
citation

' M. Rabus,  A. Fiory,  N. Ravindra,  P. Frisella,  A. Agarwal,  T. Sorsch,  J. Miner,  E. Ferry,  F. Klemens,  R. Cirelli,  W. Mansfield, "Rapid thermal processing of silicon wafers with emissivity patterns." Journal of Electronic Materials, 2006.'

In [11]:
publist[pubsource]["collection"]["name"]

'publications'

In [12]:
venue

'Journal of Electronic Materials'

In [13]:
b["title"].encode().decode('latex').replace('{', '').replace('}', '')

'Rapid thermal processing of silicon wafers with emissivity patterns'